# STA 220 Assignment 2
# Jake Tierney, Student ID: 913235409

Due __Februrary 9, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on you proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. If test cases are given, your solution must be in the same format. 
5. The total number of points is 10.

__Exercise 1__

We will compute the [PageRank](https://en.wikipedia.org/wiki/PageRank) of the articles of the [Sinhala](https://en.wikipedia.org/wiki/Sinhala_language) wikipedia, which is available at [si.wikipedia.org](https://si.wikipedia.org/wiki/%E0%B6%B8%E0%B7%94%E0%B6%BD%E0%B7%8A_%E0%B6%B4%E0%B7%92%E0%B6%A7%E0%B7%94%E0%B7%80). Additional information of the Sinhala wiki can be found [here](https://meta.wikimedia.org/wiki/List_of_Wikipedias). 

_Hints: If you don't speak Sinhalese, you might want to learn the wiki logic from the english wikipedia, and translate your findings. Also, caching is highly recommended._

__(a)__ Use the special [AllPages](https://si.wikipedia.org/wiki/%E0%B7%80%E0%B7%92%E0%B7%81%E0%B7%9A%E0%B7%82:%E0%B7%83%E0%B7%92%E0%B6%BA%E0%B7%85%E0%B7%94_%E0%B6%B4%E0%B7%92%E0%B6%A7%E0%B7%94) page and understand its logic to retrieve the url of all articles in the sinhalese wikipedia. Make sure to skip redirections.

_How many articles are there?_

In [1]:
# Importing requisite modules
import time
import requests as rq
import numpy as np
import pandas as pd
import json as js
import lxml.html as lx
from bs4 import BeautifulSoup

In [2]:
# Exercise 1, Problem a

# Defining a function that will return the list of URLs
def get_links(initial_url, final_list):

    # Set the parameters
    params = {
        "action": "query",
        "format": "json",
        "prop": "links",
        "list": "allpages",
        "aplimit": "max",
    }

    # Creating a loop that will iterate over all of the AllPages pages
    for r in range(1, 150):
        
        # If/Else statement that uses the initial URL for the first loop, and then an updated URL for subsequent loops
        if r == 1:
            response = rq.get(initial_url, params = params)
        else:
            response = rq.get(next_page, params = params)

        # Parse the HTML content
        data = BeautifulSoup(response.content, 'html.parser')

        # Find all links within the page
        links = data.find_all('a')

        # Extract link URLs and classes in a list
        link_info = []
        for link in links:
            link_url = link.get('href')
            link_class = link.get('class')

            # Append link URL and class to the result list
            link_info.append({
                'url': link_url,
                'class': link_class
            })

        # Set the next page link
        for i in range(0, len(link_info)):
            if r == 1 and link_info[i] == {'url': 'javascript:print();', 'class': None}:
                nav_index = i + 3
                next_page = "https://si.wikipedia.org" + link_info[nav_index]['url']
                break

            elif r != 1 and link_info[i] == {'url': 'javascript:print();', 'class': None}:
                nav_index = i + 4
                next_page = "https://si.wikipedia.org" + link_info[nav_index]['url']
                break

            elif '>මීළඟ පිටුව' not in str(links):
                next_page = "NA"

        # Keep only instances of main article links
        for i in range(0, len(link_info)):
            if r == 1 and link_info[i] == {'url': 'https://foundation.wikimedia.org/wiki/Special:MyLanguage/Policy:Privacy_policy', 'class': None}:
                last_article_index = i - 2
                break

            elif r != 1 and link_info[i] == {'url': 'https://foundation.wikimedia.org/wiki/Special:MyLanguage/Policy:Privacy_policy', 'class': None}:
                last_article_index = i - 3
                break

        link_info = link_info[nav_index + 1:last_article_index]
        
        main_articles = []
        
        for i in link_info:
            if i['class'] == None:
                main_articles.append(i)

        # Creating final list of URLs
        for i in main_articles:
            wiki_link = "https://si.wikipedia.org" + i['url']
            final_list.append(wiki_link)

        # Creating a break statement if there is no next page
        if next_page == "NA":
            break

        # Resting briefly to not make too many requests per minute; aiming for 25 requests per second
        time.sleep(0.04)

    return final_list


# Set the initial URL and final list inputs
initial_url = "https://si.wikipedia.org/wiki/Special:AllPages"
final_list = []

# Run the function with these inputs
wiki_pages = get_links(initial_url, final_list)

# Count the length of the output list of Wiki URLs
len(wiki_pages)

24227

In [3]:
# Confirm that this list contains only unique URLs
len(np.unique(np.array(wiki_pages)))

24227

SOLUTION: There are roughly 24,227 non-redirect article links within the Sinhala Wikipedia AllPages.

__(b, i)__ Scan all articles in the sinhalese wikipedia and retrieve all links to other articles. Avoid links to special pages, images or the ones that point to another website. Only count the proper article for links that point to a specific section. Use regular expressions to manage these cases. 
__(ii)__ Make sure to match redirections to their correct destiation article. To this end, find how wikipedia treats redirections and retrieve the true article. _(Help: Try searching for 'uc davis' on en.wikipedia.org')_
__(iii)__ Use threading to request all articles and obtain all links to other articles. _(Attention: This takes about thirty minutes!)_


_How many links to other articles are there?_

In [4]:
# Exercise 1, Problem b

# (i)

# Defining the function that will return a list of all wiki URLs referenced in the wiki_pages list
def page_urls(input_url_list):

    # Initializing the empty article_links list
    article_links = []
    
    #Iterating over every url in the wiki_pages list
    for p in input_url_list:

        # Making the API request for the given URL
        response = rq.get(p)

        # Parsing html content
        data = BeautifulSoup(response.content, 'html.parser')
       
        # Finding the body content section by class
        content_section = data.find('div', class_='mw-content-ltr mw-parser-output')
        
        # Checking if content_section is found
        if content_section:
            # Finding all links within the content section
            links = content_section.find_all('a')

            # Extracting within-page link URLs and classes
            link_info = []
            for link in links:
                link_url = link.get('href')
                link_class = link.get('class')

                # Append link URL and class to the result list
                link_info.append({
                    'url': link_url,
                    'class': link_class
                })

        # Removing any external or special links
        for i in range(0, len(link_info)):
            crit1 = '/wiki/' in str(link_info[i]['url'])
            crit2 = str(link_info[i]['class']) in ['None', "['mw-redirect']"]
            crit3 = 'https:' not in str(link_info[i]['url'])
            crit4 = 'Help:Authority_control' not in str(link_info[i]['url'])
            crit5 = 'Special:Search' not in str(link_info[i]['url'])
            if crit1 and crit2 and crit3 and crit4 and crit5:
                article_links.append(str(link_info[i]))

        # Resting briefly to not make too many requests per minute; aiming for 25 requests per second
        time.sleep(0.04)

    # Creating a list of only the unique article links
    article_links = np.unique(np.array(article_links))

    return article_links

within_article_links = page_urls(wiki_pages)

In [6]:
len(within_article_links)

34539

In [ ]:
# Exercise 1, Problem b

# (ii)

# Splitting the links found in part (i) into redirect and main article lists
redirects = []
main_articles = []
for i in within_article_links:
    if 'mw-redirect' in i:
        redirects.append(i)
    else:
        main_articles.append(i)

# Finding the destination URL for each of the redirect links
destination_urls = []
for i in range(0, len(redirects)):
    input_url = "https://si.wikipedia.org" + redirects[i][redirects[i].find("/wiki/"):redirects[i].find("class") - 4]
    response = rq.get(input_url, allow_redirects = False)
    data = response.text
    destination_url = "https://si.wikipedia.org" + data[data.find("/wiki/"):data.find("wgCentralAuthMobileDomain") - 3]
    destination_urls.append(destination_url)
    time.sleep(0.04)
    
destination_urls


In [97]:
# Exercise 1, Problem b

# (ii)

input_url = "https://si.wikipedia.org" + redirects[6][redirects[6].find("/wiki/"):redirects[6].find("class") - 4]
response = rq.get(input_url, allow_redirects = False)
data = response.text
destination_url = "https://si.wikipedia.org" + data[data.find("wgInternalRedirectTargetUrl") + 30:data.find("wgCentralAuthMobileDomain") - 3]
destination_url

'https://si.wikipedia.org/wiki/%E0%B6%87%E0%B6%BD%E0%B7%8A%E0%B7%86%E0%B7%8F_%E0%B6%9A%E0%B7%8A%E0%B7%82%E0%B6%BA%E0%B7%80%E0%B7%93%E0%B6%B8%E0%B7%8A'

In [95]:
data[data.find("wgInternalRedirectTargetUrl") + 31:data.find("wgCentralAuthMobileDomain") - 3]


'wiki/%E0%B6%87%E0%B6%BD%E0%B7%8A%E0%B7%86%E0%B7%8F_%E0%B6%9A%E0%B7%8A%E0%B7%82%E0%B6%BA%E0%B7%80%E0%B7%93%E0%B6%B8%E0%B7%8A'

In [65]:
len(redirects) + len(main_articles)



34539

In [100]:
len(redirects)




7459

In [108]:
input_url = "https://si.wikipedia.org" + redirects[3][redirects[3].find("/wiki/"):redirects[3].find("class") - 4]
response = rq.get(input_url, allow_redirects = False)
data = response.text
destination_url = "https://si.wikipedia.org" + data[data.find("/wiki/"):data.find("wgCentralAuthMobileDomain") - 3]
destination_url

'https://si.wikipedia.org/wiki/%E0%B7%83%E0%B6%BB%E0%B7%83%E0%B7%80%E0%B7%92%E0%B6%BA_%E0%B7%84%E0%B7%9C%E0%B6%B3%E0%B6%B8_%E0%B6%A0%E0%B7%92%E0%B6%AD%E0%B7%8A%E2%80%8D%E0%B6%BB%E0%B6%B4%E0%B6%A7%E0%B6%BA'

__(c)__ Compute the transition matrix (see [here](https://en.wikipedia.org/wiki/Google_matrix) and [here](https://www.amsi.org.au/teacher_modules/pdfs/Maths_delivers/Pagerank5.pdf) for step-by-step instructions). Make sure to tread dangling nodes. You may want to use: 
```
from scipy.sparse import csr_matrix
```

__(d, i)__ Set the damping factor to `0.85` and compute the PageRank for each article, using fourty iterations and starting with a vector with equal entries. __(ii)__ Obtain the top ten articles in terms of PageRank, and, retrieving the articles again, find the correponding English article, if available. 

_Return the corresponding English article titles of the top ten articles from the Sinhalese wikipedia._